In [ ]:
import cv2
from ultralytics import YOLO
# Eğitilmiş modeli yükle
model = YOLO("runs/detect/train/weights/best.pt")

# Görüntüyü yükle ve tahmin yap
image_path = "test_img.jpg"
results = model.predict(image_path, conf=0.5)

# Orijinal görüntüyü oku ve 500x500 piksele yeniden boyutlandır
original_image = cv2.imread(image_path)
resized_image = cv2.resize(original_image, (500, 500), interpolation=cv2.INTER_AREA)

# Tahmin sonuçlarını işle ve bounding box'ları çiz
for result in results:
    boxes = result.boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])
        label = result.names[class_id]
        confidence = float(box.conf[0])
        
        # Bounding box'ları yeniden boyutlandırılmış görüntüye göre ölçekle
        height, width = original_image.shape[:2]
        scale_x = 500 / width
        scale_y = 500 / height
        x1_resized = int(x1 * scale_x)
        y1_resized = int(y1 * scale_y)
        x2_resized = int(x2 * scale_x)
        y2_resized = int(y2 * scale_y)
        
        # Kutu ve etiketi çiz
        cv2.rectangle(resized_image, (x1_resized, y1_resized), (x2_resized, y2_resized), (0, 255, 0), 2)
        cv2.putText(resized_image, f"{label} {confidence:.2f}", (x1_resized, y1_resized - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Sonucu göster ve kaydet
cv2.imshow("500x500 Tahmin Sonucu", resized_image)
cv2.imwrite("output_500x500.jpg", resized_image)  # Görüntüyü 500x500 olarak kaydet
cv2.waitKey(0)
cv2.destroyAllWindows()